#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

In [69]:
import pandas as pd
import numpy as np
from scipy import stats



In [70]:
flights = pd.read_csv('/Users/vickichen/Desktop/flight/midterm-project/data/flights_without_outlier.csv')

In [71]:
data = flights[['fl_date','origin_city_name','dest_city_name','dep_delay','arr_delay','air_time','distance']]
data.head()

,fl_date,origin_city_name,dest_city_name,dep_delay,arr_delay,air_time,distance
0,2019-04-20,"Chicago, IL","Dallas/Fort Worth, TX",-8.0,-35.0,101.0,801
1,2018-11-20,"Dallas/Fort Worth, TX","Indianapolis, IN",-1.0,13.0,102.0,761
2,2018-08-28,"New York, NY","Washington, DC",-6.0,-34.0,51.0,228
3,2018-10-30,"Salt Lake City, UT","Sacramento, CA",-2.0,-7.0,81.0,532
4,2018-03-31,"Cleveland, OH","Atlanta, GA",-2.0,3.0,83.0,554


In [72]:
#Calculate speed = distance /time
dis = data['distance']
time = data['air_time']
data['avg_speed'] = dis/time * 60 #per hour

/var/folders/fz/ntgp3y9j3p777vzmhx_1fmwr0000gn/T/ipykernel_13785/602229004.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['avg_speed'] = dis/time * 60 #per hour


In [73]:
data

,fl_date,origin_city_name,dest_city_name,dep_delay,arr_delay,air_time,distance,avg_speed
0,2019-04-20,"Chicago, IL","Dallas/Fort Worth, TX",-8.0,-35.0,101.0,801,475.841584
1,2018-11-20,"Dallas/Fort Worth, TX","Indianapolis, IN",-1.0,13.0,102.0,761,447.647059
2,2018-08-28,"New York, NY","Washington, DC",-6.0,-34.0,51.0,228,268.235294
3,2018-10-30,"Salt Lake City, UT","Sacramento, CA",-2.0,-7.0,81.0,532,394.074074
4,2018-03-31,"Cleveland, OH","Atlanta, GA",-2.0,3.0,83.0,554,400.481928
...,...,...,...,...,...,...,...,...
168920,2019-06-20,"Houston, TX","Louisville, KY",-8.0,-20.0,106.0,788,446.037736
168921,2019-06-25,"Las Vegas, NV","Santa Ana, CA",0.0,0.0,44.0,226,308.181818
168922,2018-01-10,"Moline, IL","Atlanta, GA",-9.0,-5.0,91.0,633,417.362637
168923,2019-11-19,"Newark, NJ","Detroit, MI",-9.0,-23.0,80.0,488,366.000000


In [75]:
#Lets find all avg_speed with the departure delays flights, when dep_delay > 0, avg_speed with departure delays 

dep_delay_above0 = data.loc[(data['dep_delay'] >0),'avg_speed']
dep_delay_above0
dep_delay_above0.describe()

count    39713.000000
mean       408.763511
std         72.761397
min         46.391753
25%        365.000000
50%        415.555556
75%        458.102190
max        611.538462
Name: avg_speed, dtype: float64

In [76]:
#Lets find all avg_speed with the departure delays equals to 0, no departure delay fights ave_speed
dep_delay_0 = data.loc[(data['dep_delay'] == 0),'avg_speed']

In [77]:
dep_delay_0.describe()

count    9786.000000
mean      407.933197
std        70.735007
min       109.756098
25%       366.285714
50%       414.915254
75%       455.675676
max       628.767123
Name: avg_speed, dtype: float64

In [83]:
#Lets test if there is any  extreme outliers need to be removed before hypothesis.  

def remove_outliers(data,m=3):
    return data[abs(data-np.mean(data))<m*np.std(data)]



In [85]:
remove_outliers_dep_delay_above0 = remove_outliers(dep_delay_above0,m=3)
remove_outliers_dep_delay_0 = remove_outliers(dep_delay_0,m=2)

remove_outliers_dep_delay_above0.mean(),remove_outliers_dep_delay_0.mean() #from below we can see means are slightly changed. 

(409.94193264485443, 413.86366519824634)

## T test (Independent) hypothesis test 
### H0 =  Planes do not fly faster when there is the departure delay vs  Ha = Planes do fly faster when there is the the departure delay


In [91]:
# T - Test for the mean of two independent, Lets n = 20,000(sample size)
from scipy.stats import ttest_ind
t_statistic,p_value = ttest_ind(remove_outliers_dep_delay_above0[:20000],remove_outliers_dep_delay_0[:20000])

t_statistic, p_value


/var/folders/fz/ntgp3y9j3p777vzmhx_1fmwr0000gn/T/ipykernel_13785/1583684962.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  t_statistic,p_value = ttest_ind(remove_outliers_dep_delay_above0[:20000],remove_outliers_dep_delay_0[:20000])


(-4.333014137066257, 1.4757049507029225e-05)

## P = 1.4757 >0.5, We Reject null Hypothesis. 
## Conclusion: Departure delay flights on average fly faster
